In [1]:
# Many figures on the Internet tend to show 
# how greatly the S&P 500 has risen up since ancient time.
# As each of us starts investing in different times,
# the relation between starting time and return would be more important.

# This program draws;
# a. the relation between
#    (X) the acquision date, and
#    (Y) the annualized return as of the latest date
# b. the relation between
#    (X) the starting time of dollar cost avaraging strategy, and
#    (Y) the annualized return as of the latest date

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# interval '1d' gets older data than '1mo'
df_SP500 = yf.download(['^GSPC'],interval='1d',period='max')

# calculate monthly
df_SP500['year'] = df_SP500.index.year
df_SP500['month'] = df_SP500.index.month

df_SP500.sort_index(inplace=True)
df_SP500.drop_duplicates(['year','month'],inplace=True)

df_SP500['gap_month'] = range(len(df_SP500)-1,-1,-1)

[*********************100%***********************]  1 of 1 completed


a.

In [4]:
latest_close = df_SP500['Close'][-1]
# df_SP500['a_return'] = latest_close / df_SP500['Close'] - 1
df_SP500['a_return_annualized'] = (latest_close / df_SP500['Close']) ** (12/df_SP500['gap_month']) - 1

In [5]:
latest_date = df_SP500.index[-1].strftime('%Y/%m/%d')

fig = plt.figure(figsize=(20,10))
plt.style.use('dark_background')
plt.rcParams['font.size'] = 15
# plt.xlim(0, 21)
plt.ylim(-10, 30)
plt.xlabel("Acquisition Month")
plt.ylabel(f"Annualized Return [%]")
# plt.xticks(ticks=range(21), labels=range(21))
# plt.yticks(ticks=[i*10000 for i in range(4+1)], labels=[f"{i*10000:,}" for i in range(4+1)])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.title(f"Acquisition Month and Annualized Return of S&P 500\n(as of {latest_date})")

plt.plot(df_SP500['a_return_annualized']*100,linestyle="-",color='white',linewidth=2)
for _x in df_SP500.index[df_SP500['month']==1]:
    plt.axvline(x=_x,color="grey",linewidth=0.5)
for _x in df_SP500.index[(df_SP500['month']==1) & (df_SP500['year']%10==0)]:
    plt.axvline(x=_x,color="white",linewidth=0.5)
for _y in np.arange(-10,30+1,5):
    plt.axhline(y=_y,color="grey",linewidth=0.5)
plt.axhline(y=0,color="white",linewidth=1)

fig.savefig("../output/20230703_Start_and_Return_a.png")
plt.close()


b.

In [6]:
constant = 10000
df_SP500['b_quantity'] = constant / df_SP500['Close']
df_SP500.at[df_SP500.index[-1],'b_quantity'] = 0
df_SP500['b_quantity_cumsum'] = df_SP500.loc[::-1,'b_quantity'].cumsum()[::-1]
df_SP500['b_balance'] = latest_close * df_SP500['b_quantity_cumsum']
df_SP500['b_cost'] = df_SP500['gap_month']*constant

df_SP500['b_return_annualized'] = (df_SP500['b_balance'] / df_SP500['b_cost']) ** (12/df_SP500['gap_month']) - 1
df_SP500['b_return_annualized'].fillna(0,inplace=True)

In [7]:
fig = plt.figure(figsize=(20,10))
plt.style.use('dark_background')
plt.rcParams['font.size'] = 15
# plt.xlim(0, 21)
plt.ylim(0, 20)
plt.xlabel("Starting Month")
plt.ylabel(f"Annualized Return [%]")
# plt.xticks(ticks=range(21), labels=range(21))
plt.yticks(ticks=np.arange(0,21,2), labels=[f"{i:,}" for i in np.arange(0,21,2)])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.title(f"Dollar Cost Strategy Starting Month and Annualized Return of S&P 500\n(as of {latest_date})")

plt.plot(df_SP500['b_return_annualized']*100,linestyle="-",color='white',linewidth=2)
for _x in df_SP500.index[df_SP500['month']==1]:
    plt.axvline(x=_x,color="grey",linewidth=0.5)
for _x in df_SP500.index[(df_SP500['month']==1) & (df_SP500['year']%10==0)]:
    plt.axvline(x=_x,color="white",linewidth=0.5)
for _y in np.arange(0,20+1,2):
    plt.axhline(y=_y,color="grey",linewidth=0.5)
plt.axhline(y=0,color="white",linewidth=1)
fig.savefig("../output/20230703_Start_and_Return_b_DCA.png")
plt.close()
